In [1]:
import faiss

In [2]:
import numpy as np 

In [ ]:
# Xt = np.loadtxt('../zebala/saved_db_5m.csv', delimiter = ',')

In [3]:
rng = np.random.default_rng(200)
vectors = rng.random((10**5, 70), dtype=np.float32)

vectors.shape

(100000, 70)

In [4]:
data = vectors.copy()
data[:1,:]

array([[0.03806317, 0.64683414, 0.23583442, 0.6639199 , 0.33604568,
        0.02990162, 0.5436652 , 0.17783129, 0.89160764, 0.57349664,
        0.43597007, 0.551164  , 0.47267753, 0.52692044, 0.47771925,
        0.37424278, 0.7041597 , 0.06079406, 0.67475194, 0.36999255,
        0.44853985, 0.3575145 , 0.69858366, 0.241557  , 0.62255454,
        0.044173  , 0.00221592, 0.90875757, 0.7624761 , 0.52673286,
        0.8637938 , 0.11358559, 0.6735863 , 0.14124113, 0.79021454,
        0.44684768, 0.07083786, 0.50149125, 0.80701876, 0.78536385,
        0.22534001, 0.04118073, 0.6816156 , 0.2719401 , 0.01268417,
        0.8496885 , 0.8404332 , 0.9196788 , 0.8326662 , 0.56165546,
        0.7264039 , 0.26703602, 0.6613052 , 0.7362026 , 0.6181503 ,
        0.03443956, 0.76080143, 0.6502585 , 0.7003726 , 0.18064976,
        0.825382  , 0.45629853, 0.74387693, 0.6759924 , 0.6282045 ,
        0.8709608 , 0.03340232, 0.7640089 , 0.34944552, 0.74151176]],
      dtype=float32)

## Trial 1 : Using IndexFlatL2 as coarse quantizer and IVFPQ 

In [43]:
del index

NameError: name 'index' is not defined

In [69]:
quantizer = faiss.IndexFlatL2(data.shape[1])

index = faiss.IndexIVFPQ(quantizer,70,100,7,8) #=> Quantizer , Dimension , clusters , reduceDimension , nbits

In [70]:
rng = np.random.default_rng(100)
Xtrain = rng.random((2500, 70), dtype=np.float32)

In [71]:
index.train(Xtrain)

In [72]:
index.add(data)

In [73]:
rng = np.random.default_rng(100)
Q = rng.random((1,70), dtype=np.float32)
Q

array([[0.7668845 , 0.8349816 , 0.12417817, 0.596554  , 0.08026338,
        0.28886324, 0.44347   , 0.04295152, 0.58785254, 0.9736544 ,
        0.947227  , 0.59647167, 0.4118786 , 0.7902631 , 0.924844  ,
        0.91033936, 0.00349164, 0.6881544 , 0.7191256 , 0.18999141,
        0.50552726, 0.9814789 , 0.687205  , 0.28474003, 0.3224274 ,
        0.6292731 , 0.14462936, 0.58103645, 0.57213193, 0.5999122 ,
        0.7742522 , 0.5352481 , 0.7242624 , 0.9957771 , 0.1285438 ,
        0.50194603, 0.12643135, 0.77102256, 0.14178151, 0.49416173,
        0.22792953, 0.99767447, 0.43094128, 0.9786219 , 0.88263243,
        0.39356804, 0.44872624, 0.32192522, 0.8011627 , 0.86218065,
        0.48825973, 0.79932487, 0.9635074 , 0.6914306 , 0.7044168 ,
        0.40851784, 0.9652947 , 0.38976508, 0.64390755, 0.13164836,
        0.6317189 , 0.62549746, 0.17958134, 0.08240134, 0.82947433,
        0.27457297, 0.8404267 , 0.656117  , 0.11085582, 0.01467597]],
      dtype=float32)

In [74]:
%%time

dist_x , id_x = index.search(Q,5)

dist_x

CPU times: total: 0 ns
Wall time: 1.48 ms


array([[5.5443997, 5.7065644, 5.729079 , 5.779186 , 5.8386874]],
      dtype=float32)

In [ ]:
faiss.write_index(index, "index_ivfpq_100k.index")

## Trial 2 : Using hnsw as a quantizer with IVFPQ

In [ ]:
del quantizer_hnsw

In [ ]:
quantizer_hnsw = faiss.IndexHNSWFlat(70,30)
# index_hnswivfpq = faiss.IndexIVFPQ(quantizer_hnsw, 70, 1000, 35, 8)

In [ ]:
%time
# quantizer_hnsw.train(Xtrain)
quantizer_hnsw.add(data)

In [ ]:
%%time
index_hnswivfpq.train(Xtrain)
index_hnswivfpq.add(data)

In [ ]:
%time dists , ids = quantizer_hnsw.search(Q,10)

dists

In [ ]:
Q

## Statistics and Comparison between cosine similarity of Q and the vectors in data  

### Brute-Force cos-similarity 

In [ ]:
b = Q[0]
a3la_cos_sim = []
for idx in range(90000):
    a = data[idx,:].reshape(1,-1)
    xos = np.dot(a, b)/(np.linalg.norm(a)*(np.linalg.norm(b)))
    if xos > 0.85:
        a3la_cos_sim.append(xos)
        print(xos, idx)

In [ ]:
np.max(a3la_cos_sim)

## Ids extracted from faiss hnswivfpq as nearest distance

In [ ]:
print(id_x) #=> ivfpq no hnsw
b = Q[0]
for idx in id_x[0]:
    a = data[idx,:].reshape(1,-1)
    print(np.dot(a, b)/(np.linalg.norm(a)*(np.linalg.norm(b))))

In [ ]:
print(ids) #=> with hnsw

b = Q[0]
for idx in ids[0]:
    a = data[idx,:].reshape(1,-1)
    print(np.dot(a, b)/(np.linalg.norm(a)*(np.linalg.norm(b))))

## PQ

In [ ]:
import nanopq

In [ ]:
N, Nt, D = 10000, 5000, 70
# X = np.random.random((N, D)).astype(np.float32)  # 10,000 128-dim vectors to be indexed
Xt = np.random.random((Nt, D)).astype(np.float32)  # 2,000 128-dim vectors for training
query = np.random.random((D,)).astype(np.float32)  # a 128-dim query vector

# Instantiate with M=8 sub-spaces
pq = nanopq.PQ(M=7)

# Train codewords
pq.fit(Xt)

# Encode to PQ-codes
X_code = pq.encode(data.astype(np.float32))  # (90000, 7) with dtype=np.uint8

# Results: create a distance table online, and compute Asymmetric Distance to each PQ-code 
dists = pq.dtable(query).adist(X_code)  # (90000, ) 

In [ ]:
dists

## IVF_Flat    WINNER WINNER CHICKEN DINNER

In [ ]:
from memory_profiler import memory_usage


def memory_usage_run_queries(args):
    # This part is added to calcauate the RAM usage
    mem_before = max(memory_usage())
    mem = memory_usage(proc=(run_queries, args, {}), interval = 1e-3)
    return max(mem) - mem_before




In [ ]:
# Generating random data (1M vectors with dimension 70)
np.random.seed(42)
Xt = np.random.rand(10000, 70).astype('float32')

# Parameters
d = 70  # Dimension of the vectors
nlist = 100  # Number of clusters
quantizer = faiss.IndexFlatL2(d)  # Flat index for clustering

# Create IndexIVFFlat
index = faiss.IndexIVFFlat(quantizer, d, nlist, faiss.METRIC_L2)

index.train(Xt)
index.add(data)

# Save the index
# faiss.write_index(index, "index_ivf_flat.index")


In [ ]:
def run_queries(Q,K):
    dists_flat , ids_flat = index.search(Q,K)
    return dists_flat, ids_flat

memory_usage_run_queries((Q,40))

In [ ]:
dists_flat # L2 distance 

In [ ]:
b = Q[0] #cosine similarity
for idx in ids_flat[0]:
    a = data[idx,:].reshape(1,-1)
    print(np.dot(a, b)/(np.linalg.norm(a)*(np.linalg.norm(b))))

## Index IVFPQ only

In [ ]:
# Generating random data (1M vectors with dimension 70)
np.random.seed(42)
Xt = np.random.rand(10000, 70).astype('float32')

# Parameters
d = 70  # Dimension of the vectors
nlist = 100  # Number of clusters
m = 7  # Number of subquantizers
nbits = 8  # Number of bits for quantization

# Create IndexIVFPQ
index = faiss.IndexIVFPQ(faiss.IndexFlatL2(d), d, nlist, m, nbits)

# Train the index
index.train(Xt)
index.add(data)

dists_ivfpq , ids_ivfpq = index.search(Q,5)



# Save the index
#faiss.write_index(index, "index_ivf_pq.index")


In [ ]:
dists_ivfpq

In [ ]:
b = Q[0]
for idx in ids_ivfpq[0]:
    a = data[idx,:].reshape(1,-1)
    print(np.dot(a, b)/(np.linalg.norm(a)*(np.linalg.norm(b))))

## IVF only 

In [ ]:
# Generating random data (1M vectors with dimension 70)
np.random.seed(42)
Xt = np.random.rand(2000, 70).astype('float32')

# Parameters
d = 70  # Dimension of the vectors
nlist = 100  # Number of clusters
quantizer = faiss.IndexFlatL2(d)  # Flat index for clustering

# Create IndexIVF
index = faiss.IndexIVF(quantizer, d, nlist, faiss.METRIC_L2)

# Train the index
index.train(Xt)
index.add(data)

# Save the index
# faiss.write_index(index, "index_ivf.index")
